In [29]:
import torch
from torch.autograd import Variable


In [30]:
# Declaring a Variable #
x= Variable(torch.ones(2,2)*2,requires_grad=True)
z=2*(x*x)+5*x
# compute dz/dx #
z.backward(torch.ones(2,2))
print(x.grad.data)


 13  13
 13  13
[torch.FloatTensor of size 2x2]



In [31]:
# Building a fully connected 4 layer neural network #

# Input layer constists of 784 nodes, 1st hidden layer of 100 nodes,
# 2nd hidden layer of 200, and the output layer of 10 nodes
#  Using ReLU function and softmac output layer


# First step consists of nuilding the model is to use the base class
# nn.Module to use all of its functionality and overwrite capabilities 
# for model construction and forward pass

import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        super(Net,self).__init__()
        # Input Layer
        # A fully connectd network is represented by nn.Linear object
        # first argument beeing the nimber of nodes in layer l, second 
        # beeing the number of layers in l+1
        self.fc1=nn.Linear(28*28,200)
        # First Hidden Layer
        self.fc2=nn.Linear(200,200)
        # Second Hidden Layer
        self.fc3=nn.Linear(200,10)
        
    # Defining how the data flows through the network with the forward 
    # function
    def forward(self,x):
        # Feed the input data to fully connected layer 
        # and then calculate the activation function
        x=F.relu(self.fc1(x))
        #pass to the 2nd layer
        x=F.relu(self.fc2(x))
        #pass to the 3rd layer
        x=F.relu(self.fc3(x))
        # In the last layer, Insted of ReLU return the log 
        # softmax activation This, combined with the negative log 
        # likelihood loss function which will be defined later, 
        # gives us a multi-class cross entropy based loss function 
        # which we will use to train the network.
        return F.log_softmax(x)


        



In [32]:
# Create  instance of class Net and see its charcteristics #
net=Net()
print(net)

Net(
  (fc1): Linear(in_features=784, out_features=200)
  (fc2): Linear(in_features=200, out_features=200)
  (fc3): Linear(in_features=200, out_features=10)
)


In [33]:
# Training the network #
import torch.optim as optim 
from torchvision import datasets, transforms

learning_rate=1e-2

# Create a stochastic gradient descent optimizer. First parameter is the 
# parameters of the optimizer which are provided by the .parameters() method
# provided by the nn.Module class. 
optimizer=optim.SGD(net.parameters(),lr=learning_rate,momentum=0.9)
# Create a loss function. In this case, the  negative log likehood 
# loss.
criterion=nn.NLLLoss()


batch_size, epochs, log_interval=200, 10, 10


In [34]:
# Downloading and creating train and test set #

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=batch_size, shuffle=True)
  
test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
batch_size=batch_size, shuffle=True)

In [35]:
# Run the training loop #
for epoch in range(epochs):
    for batch_idx,(data,target) in enumerate(train_loader):
        # Convert data and and target into Variables
        data,target=Variable(data),Variable(target)
        # Resize data from (batch_size,1,28,28) to (batch_size,28*28), with 
        # view(-1,28*28) we say that the second dimension 
        data=data.view(-1,28*28)
        
        # Reset all the gradients to zero for the next back propagation 
        optimizer.zero_grad()
        
        # We pass the input data to the model. This will call the forward()
        # method in the Net class. After this line is run, the variable net_out 
        # will now hold the log softmax output of our neural network for the given data batch.
        net_out=net(data)
        # We get the negative log likelihood loss between the output of our
        # network and our target batch data
        loss=criterion(net_out,target)
        
        # Run back propagation
        loss.backward()
        # Execute gradient descent step based on the gradients 
        # calculated during the backward operation
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        

D:\Users\papagian\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train Epoch: 0 [0/60000 (0%)]	Loss: 2.308497
Train Epoch: 0 [200/60000 (0%)]	Loss: 2.311570
Train Epoch: 0 [400/60000 (1%)]	Loss: 2.304468
Train Epoch: 0 [600/60000 (1%)]	Loss: 2.300134
Train Epoch: 0 [800/60000 (1%)]	Loss: 2.287208
Train Epoch: 0 [1000/60000 (2%)]	Loss: 2.283334
Train Epoch: 0 [1200/60000 (2%)]	Loss: 2.282429
Train Epoch: 0 [1400/60000 (2%)]	Loss: 2.261555
Train Epoch: 0 [1600/60000 (3%)]	Loss: 2.244309
Train Epoch: 0 [1800/60000 (3%)]	Loss: 2.250689
Train Epoch: 0 [2000/60000 (3%)]	Loss: 2.216508
Train Epoch: 0 [2200/60000 (4%)]	Loss: 2.215532
Train Epoch: 0 [2400/60000 (4%)]	Loss: 2.185968
Train Epoch: 0 [2600/60000 (4%)]	Loss: 2.184533
Train Epoch: 0 [2800/60000 (5%)]	Loss: 2.173451
Train Epoch: 0 [3000/60000 (5%)]	Loss: 2.141259
Train Epoch: 0 [3200/60000 (5%)]	Loss: 2.132670
Train Epoch: 0 [3400/60000 (6%)]	Loss: 2.122221
Train Epoch: 0 [3600/60000 (6%)]	Loss: 2.114306
Train Epoch: 0 [3800/60000 (6%)]	Loss: 2.037918
Train Epoch: 0 [4000/60000 (7%)]	Loss: 1.999659

In [36]:
# Run a test loop
test_loss=0
correct=0
for data,target in test_loader:
    data,target=Variable(data),Variable(target)
    data=data.view(-1,28*28)
    net_out=net(data)
    # sum up batch loss
    test_loss+=criterion(net_out,target).data[0]
    #get the index of max log probability
    pred=net_out.data.max(1)[1]
    correct+=pred.eq(target.data).sum()
    
test_loss /= len(test_loader.dataset)    
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    

D:\Users\papagian\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:44: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.



Test set: Average loss: 0.0015, Accuracy: 8831/10000 (88%)



In [37]:
target

Variable containing:
 2
 8
 0
 9
 3
 5
 0
 1
 3
 6
 5
 5
 2
 8
 4
 5
 3
 8
 8
 3
 5
 1
 9
 3
 3
 9
 8
 2
 1
 5
 6
 5
 0
 8
 9
 5
 9
 1
 1
 4
 6
 8
 5
 2
 9
 8
 3
 7
 8
 9
 0
 9
 4
 2
 9
 8
 9
 7
 9
 3
 2
 8
 9
 8
 8
 5
 5
 2
 1
 7
 9
 0
 9
 8
 2
 9
 7
 1
 8
 0
 0
 8
 4
 9
 0
 3
 5
 3
 7
 9
 7
 0
 0
 0
 7
 3
 2
 9
 3
 4
 0
 1
 1
 7
 9
 8
 7
 5
 8
 7
 4
 3
 9
 1
 2
 1
 1
 3
 0
 5
 3
 5
 5
 8
 7
 7
 4
 7
 5
 7
 5
 3
 2
 2
 5
 2
 6
 7
 2
 1
 5
 0
 1
 5
 5
 6
 6
 9
 2
 3
 1
 2
 6
 7
 8
 8
 2
 9
 8
 9
 1
 3
 1
 6
 3
 4
 1
 0
 6
 2
 5
 0
 6
 1
 2
 5
 8
 1
 1
 5
 4
 3
 8
 8
 0
 3
 2
 9
 2
 8
 9
 3
 7
 6
 6
 4
 3
 0
 5
 8
[torch.LongTensor of size 200]